## Imports

In [35]:
import sagemaker as sage
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import transformer
from sagemaker import model

role = get_execution_role()

## Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our SageMaker operations.

In [36]:
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/aspect-sentiment'.format(account, region)
print(image)

model_data = ""

298841451579.dkr.ecr.us-east-1.amazonaws.com/decision-aspect-sentiment


## Create Model

In [37]:
model=sage.model.Model(model_data, image, role=role,  name='aspect-sentiment', sagemaker_session=sess) 

## Launch Endpoint and Predictor Object

In [38]:
endpoint_name = 'aspect'

In [39]:
predictor=model.deploy(1, 'ml.m4.xlarge', endpoint_name = endpoint_name)
#model.deploy(1,'ml.m4.xlarge',endpoint_name=endpoint_name) 

Using already existing model: aspect-sentiment


-*

UnexpectedStatusException: Error hosting endpoint aspect: Failed. Reason:  The repository 'bqs' does not exist in the registry with id '298841451579'..

# Invoke Endpoint

In [6]:
client = boto3.client('sagemaker-runtime')
content_type = "image/jpeg"

### Testing on a single audio file

The model takes an image file (.jpg/.png) as input. We give the location of the sample imput file as input. An example has been shown in the following code shell.

We get the response for our request as a dictionary of Barcode Data and the Barcode Type associated with the barcode. In case of a QR code, the Barcode Type is returned as "QR-code".

#### Preparing the input file for payload

In [7]:
file_name = 'data/sample_files/QRcode_example5.jpg'

with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
    ne = open('n.txt','wb')
    ne.write(b)

#### Reading the response

In [9]:
%%time
response = client.invoke_endpoint(
    EndpointName = endpoint_name, 
    ContentType = content_type,
    Body=b
)
print(response['Body'].read())

b'{"upi://pay?aid=uGICAgICAg_ziOw&pa=nishanthballal.9-1@okicici&pn=Nishanth%20Ballal": "QRCODE"}'
CPU times: user 13.6 ms, sys: 106 µs, total: 13.7 ms
Wall time: 591 ms


## Batch Transform Job

Now let's use the model built to run a batch inference job and verify it works.

### Batch Transform Input Preparation

The snippet below is removing the "label" column (column indexed at 0) and retaining the rest to be batch transform's input. NOTE: This is the same training data, which is a no-no from a statistical/ML science perspective. But the aim of this notebook is to demonstrate how things work end-to-end.

In [10]:
bucket = sess.default_bucket()

transform_input_prefix = "barcode/validation"
TRANSFORM_WORKDIR = "data/transform" 
# upload data from local directory to bucket

transform_input = sess.upload_data(TRANSFORM_WORKDIR, key_prefix=transform_input_prefix) 

print ("Transform Data Location " + transform_input)

Transform Data Location s3://sagemaker-us-east-2-428712150059/barcode/validation


### Run Batch Transform

Now that our batch transform input is setup, we run the transformation job next

In [11]:
trans = transformer.Transformer('barcode', 1, instance_type = 'ml.m4.xlarge', strategy= "SingleRecord", assemble_with=None, 
                                  output_path="s3://"+bucket+"/barcode/batch-transform-output", output_kms_key=None, accept=None, max_concurrent_transforms=None, 
                                  max_payload=None, tags=None, env=None, 
                                  base_transform_job_name=None, sagemaker_session=sess, volume_kms_key=None)

In [12]:
trans.transform(transform_input, data_type='S3Prefix', content_type="image/jpeg",
           input_filter=None, output_filter=None, join_source=None, wait=False, logs=True)

In [13]:
print("Batch Transform output saved to :" + trans.output_path)

Batch Transform output saved to :s3://sagemaker-us-east-2-428712150059/barcode/batch-transform-output


In [14]:
# Print the outputs

s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sess.default_bucket())

for object_summary in my_bucket.objects.filter(Prefix="barcode/batch-transform-output"+ "/"):
   file_name = object_summary.key.split('/')[-1]
   print(file_name)
   s3.Bucket(sess.default_bucket()).download_file("barcode/"+"batch-transform-output" +"/"+ file_name,  'batch_results')
   with open('batch_results') as f:
       results = f.readlines()  
   
   print("".join(results))

QRcode_example5.jpg.out
{"upi://pay?aid=uGICAgICAg_ziOw&pa=nishanthballal.9-1@okicici&pn=Nishanth%20Ballal": "QRCODE"}
barcode_example.png.out
{"{\"author\": \"Adrian\", \"site\": \"PyImageSearch\"}": "QRCODE", "https://www.pyimagesearch.com/": "QRCODE", "PyImageSearch": "QRCODE", "AdrianRosebrock": "CODE128"}
barcode_example3.jpg.out
{"14MF3IM01": "CODE39"}


### Clean Up Endpoint

In [ ]:
predictor.delete_endpoint()